# Task 4

In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('Task_3_and_4_Loan_Data.csv')
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [ ]:
class FicoBucketer:
    def __init__(self, num_buckets=10):
        self.num_buckets = num_buckets
        self.boundaries = []

    def _log_likelihood(self, n, k):
        if n == 0 or k == 0 or k == n:
            return 0
        p = k / n
        return k * np.log(p) + (n - k) * np.log(1 - p)

    def fit(self, fico_scores, defaults):
        x = list(defaults)
        y = list(fico_scores)
        n = len(x)

        default = [0 for _ in range(851)]
        total = [0 for _ in range(851)]

        for i in range(n):
            y[i] = int(y[i])
            default[y[i] - 300] += x[i]
            total[y[i] - 300] += 1

        for i in range(0, 551):
            if i > 0:
                default[i] += default[i - 1]
                total[i] += total[i - 1]

        r = self.num_buckets
        dp = [[[-10**18, 0] for _ in range(551)] for _ in range(r + 1)]

        for i in range(r + 1):
            for j in range(551):
                if i == 0:
                    dp[i][j][0] = 0
                else:
                    for k in range(j):
                        if total[j] == total[k]:
                            continue
                        if i == 1:
                            dp[i][j][0] = self._log_likelihood(total[j], default[j])
                        else:
                            gain = self._log_likelihood(total[j] - total[k], default[j] - default[k])
                            candidate = dp[i - 1][k][0] + gain
                            if dp[i][j][0] < candidate:
                                dp[i][j][0] = candidate
                                dp[i][j][1] = k
        k = 550
        r_copy = r
        boundaries = []
        while r_copy >= 0:
            boundaries.append(k + 300)
            k = dp[r_copy][k][1]
            r_copy -= 1

        self.boundaries = boundaries

    def get_boundaries(self):
        return self.boundaries



In [26]:
bucketer = FicoBucketer(num_buckets=10)
bucketer.fit(df['fico_score'], df['default'])

#Print boundaries (from high to low) as the problem suggests
print("Bucket boundaries:")
print(bucketer.get_boundaries())

Bucket boundaries:
[850, 753, 752, 732, 696, 649, 611, 580, 552, 520, 300]


We get meaningful splits